# Alliance Summary

Given a 6 team, two alliance match, provide a summary.


In [94]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

from common import canalytics

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Alliance Summary')

team_summary = pd.read_csv('./data/team_summary.csv', index_col=0)


Loading BokehJS ...

In [95]:
# Define the alliances to review

red = pd.Series([2158,7492,8325],index=range(0,3))
blue = pd.Series([8507,2613,8580],index=range(3,6))
match=pd.concat([red,blue])

In [96]:
#alliances = team_summary[lambda a: a.team.isin(match)]
alliances = team_summary

In [103]:
#alliances.loc[:,'alliance']='blue'
alliances.loc[lambda df: df.team.isin(red),'alliance']='red'
alliances.loc[lambda df: df.team.isin(blue),'alliance']='blue'

alliances.dropna(subset=['alliance'],inplace=True)
alliances

,team,scouted_matches,matches,valid_tba_and_scouting,scouting_points_total,climb_success_total,climb_failed_total,climb_points_total,teleop_cargo_upper_total,teleop_cargo_lower_total,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,alliance
3,2158,6,12,6,156.0,7,2,78,44.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red
4,2613,9,12,9,50.0,0,0,0,0.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blue
15,7492,6,12,6,143.0,9,0,120,21.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red
19,8325,4,12,4,6.0,2,1,12,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red
22,8507,9,12,9,53.0,5,2,30,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blue
25,8580,8,12,8,87.0,8,0,48,1.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blue


In [104]:
# - OPRs
# - Average Cargo 
# For Alliance
# - Average CLimb points
# - previous OPR ( by flag )
# - previous District Info    
alliances=alliances.round(decimals=2)
alliances=alliances.fillna(0)

alliance_summary=alliances.groupby('alliance').agg(
                    avg_climb_points = pd.NamedAgg(column='climb_points_avg',aggfunc='sum'),
                    avg_cargo = pd.NamedAgg(column='total_cargo_avg',aggfunc='sum')
)


source = ColumnDataSource(alliances)


columns = [
        TableColumn(field="alliance", title="Alliance"),
        TableColumn(field="team", title="Team"),
        TableColumn(field="oprs", title="OPRS (Event)"),
        TableColumn(field="pre_oprs", title="OPRS (Prior)"),
        TableColumn(field="total_cargo_avg", title="Total Cargo"),
        TableColumn(field="auto_cargo_avg", title="Auto Cargo"),
        TableColumn(field="teleop_cargo_avg", title="Teleop Cargo"),
        TableColumn(field="highest_endgame_position", title="Highest"),
        TableColumn(field="climb_points_avg", title="Climb Pts"),
        TableColumn(field="pre_climb_points", title="Prior Climb Pts")]

next_match_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        fit_columns=True,
        reorderable=True)

#print(f"Next match: {next_match} at predicted time: {predicted_time}\n")


In [105]:
alliance_summary

,avg_climb_points,avg_cargo
alliance,,
blue,6.5,9.27
red,17.5,13.00


In [106]:
show(next_match_table)